---
title: "Integrate and Fire Neuron"
project:
  type: website
format:
  html:
    code-fold: true
    code-tools: true
jupyter: python 3
number-sections: true
---

In this notebook we will use Python to simulate the integrate and fire (I&F) neuron model.  We'll investigate, in particular, how the spiking activity varies as we adjust the input current $I$.

#  Preliminaries
Before beginning, let's load in the Python package we'll need:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

#  Numerical solutions - Introduction

 
How do we compute a numerical solution to the integrate and fire model?
  The basic idea is to rearrange the differential equation to get $V(t+1)$ on
  the left hand side, and $V(t)$ on the right hand side.  Then, if we know
  what's happening at time $t$, we can solve for what's happening at time $t+1$.

For example, consider the differential equation:

$$
  \dfrac{dV}{dt} = \dfrac{I}{C}
$$

In words, we can think of:

$dV$ as the "change in voltage V",

$dt$ as the "change in time t".

Let's consider the case that we record the voltage $V$ in discrete time steps. So
we observe:

$V[0], V[1], V[2], \ldots$

at times:

$dt, \, 2*dt, \, 3*dt, \ldots$

where $dt$ is the time between our samples of $V$.

We can now write the "change in voltage V" as:

$$
  dV = V(t+1) - V(t)
$$

Notice that the change in voltage is the difference in V between two
sequential time samples.  Now, let's rewrite $\dfrac{dV}{dt}$ as,

$$
  \dfrac{dV}{dt} = \dfrac{ V(t+1) - V(t) }{ dt } 
$$

where we've replaced $dV$.  Now, let's substitute this expression into the equation at the top of this file:

$$
  \dfrac{ V(t+1) - V(t) }{ dt } = \dfrac{I}{C}.  
$$

Solving this equation for $V(t+1)$ you'll find  that:

$$
  V(t+1) = V(t) + dt*(I/C)
$$

Notice that, in this expression, we use our current value of the voltage V(t) and the model (I/C) to determine the next value of the voltage V(t+1).

Now, let's program this equation in Python.  First, let's set the values
for the parameters $I$ and $C$.

In [2]:
C=1.0
I=1.0

  We also need to set the value for $dt$.  This defines the time step for our
  model.  We must choose it small enough so that we don't miss anything
  interesting.  We'll choose:

In [3]:
dt=0.01

Let's assume the units of time are seconds.  So, we step forward in time by $0.01$ s.

The right hand side of our equation is nearly defined, but we're still missing one thing, $V(t)$.

::: {.callout}
**Q:**  What value do we assign to $V(t)$?
:::

So here's an easier question:  what *initial* value do we assign to $V(t)$?
 
 To start, we'll create an array of zeros to hold our results for $V$:

In [4]:
V = np.zeros([1000,1])
V.shape

(1000, 1)

This array `V` consists of 1000 rows and 1 column. We can think of each row entry as corresponding to a discrete step in time. Our goal is to fill-in the values of `V` (i.e., step forward in time), in a way consistent with our model.

Let's choose an initial value for `V` of 0.2, which in our simple model we'll assume represents the rest state.

In [5]:
V[0]=0.2

::: {.callout}
**Q:**  Given the initial state `V[0]=0.2`, calculate `V[1]`.  Then calcualte `V[2]`.
:::

  After the two calculations above, we've moved forward two time steps into
  the future, from $t=0$ s to $t=0.01$ s, and then from $t=0.01$ s to $t=0.02$ s.  But what
  if we want to know $V$ at $t=10$ s?  Then, this iteration-by-hand procedure becomes
  much too boring and error-prone.  So, what do we do?  Let's make the
  computer do it ...

# Numerical solutions - Implementation

Let's computerize this iteration-by-hand procedure to find `V[999]`.

To do so, we'll use a [for-loop](https://wiki.python.org/moin/ForLoop).

Here's what it looks like:

In [6]:
for k in range(1,999):
    V[k+1] = V[k] + dt*(I/C)

::: {.callout}
**Q:**  Does this loop make sense?  Describe what's happening here.
:::

::: {.callout}
**Q:**  Why does the `range` command end at `999`?
:::

Execute this for-loop and examine the results in vector `V`.  To do so, let's plot `V`:

In [9]:
# plt.plot(V);

::: {.callout}
**Q:**  What happens to the voltage after 1000 steps?
:::

  This plot is informative, but not great.  Really, we'd like to plot the
  voltage as a function of *time*, not steps or indices.  To do so, we
  need to define a time axis:

In [11]:
t = np.arange(0,len(V))*dt

::: {.callout}
**Q:**  What's happening in the command above?  Does it make sense?  (If not, trying printing or plotting `t`.)
:::

Now, with *time* defined, let's redo the plot of the voltage with the axes labeled appropriately.

In [15]:
# plt.plot(t,V)
# plt.xlabel('Time [s]');
# plt.ylabel('V');

Finally, let's put it all together . . .

# I&F CODE (version 1)

  So far, we constructed parts of the I&F model in bits-and-pieces.
  Let's now collect all the code, compute a numerical solution to
  the I&F model, and plot the results (with appropriate axes).

First, let's clear all the variables:

In [ ]:
%reset

In [18]:
import numpy as np
import matplotlib.pyplot as plt

I   = 1                           #Set the parameter I.
C   = 1                           #Set the parameter C.
dt  = 0.01                        #Set the timestep.
V   = np.zeros([1000,1])             #Initialize V.
V[0]= 0.2;                        #Set the initial value of V.

for k in range(1,999):            #March forward in time,
    V[k+1] = V[k] + dt*(I/C)      #... updating V along the way.

t = np.arange(0,len(V))*dt        #Define the time axis.

#plt.plot(t,V)                         #Plot the results.
#plt.xlabel('Time [s]')
#plt.ylabel('Voltage [mV]');

::: {.callout}
**Q:**  Adjust the parameter `I`.  What happens to `V` if `I=0`?  Can you set `I` so that `V` > 20 within 10 s?
:::

# Voltage threshold

  Notice, our model is missing something important:  **the reset**.

::: {.callout}
**Q:**  Without the reset, how does the voltage behave as $t\rightarrow\infty$ (if $I>0$)?
:::

 Now, let's update our model to include the reset.  To do so, we'll add two things to our code.
  
  - First, we'll define the voltage threshold `Vth`, and
  reset voltage `Vreset`.
  - Second, we'll check to see if `V` exceeds
  `Vth` using an [if-statement](https://docs.python.org/3/tutorial/controlflow.html);  if it does, then we'll set `V` equal to
  `Vreset`.

::: {.callout}
**Q:**  How will you update the code below to include the reset?
:::

In [19]:
Vth = 1;        #Define the voltage threshold.
Vreset = 0;     #Define the reset voltage.

for k in range(1,999):            #March forward in time,
    V[k+1] = V[k] + dt*(I/C)      #Update the voltage,
    ### ADD SOMETHING HERE??? --------------------------

# I&F CODE (version 2)

Now, let's put it all together to make a complete I&F model (with a thershold and reset), simulate it, and plot the result.

In [21]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [ ]:
### ADD YOUR CODE!

::: {.callout}
**Q:** Adjust the parameter `I`.  What happens to `V` if `I=10`?  If `I=100`?
:::

::: {.callout}
**Q:** Adjust the parameter `C`.  What happens to `V` if `C=0.1`?  If `C=10`?
:::

::: {.callout}
**Q:** What is "spiking" in this I&F model?
:::

# Discussion

1. Describe the main components of the IF model. Draw it (in some way).
2. Describe the main components of the LIF model. Draw it (in some way).
3. Describe the differences and similarities between the IF and LIF models.
4. The IF model is meant to mimic a neurons activity. Describe what is realistic about the IF model? What is unrealistic?
5. Describe the roles of the IF model parameters `Vreset` and `Vthreshold`.
6. Consider the IF model. Sketch voltage (V) versus time (t) for a small input current, for a large input current.
    1. How does an increase in capacitance (C) impact the dynamics?
    2. Can you interpret this physically?

# Challenges

1. Consider the *LIF* model. Plot (in Python) voltage (V) versus time (t) for a small input current, for a large input current.
2. How does an increase in the capacitance (C) impact the dynamics of the LIF model?
3. How does an increase in the resistance (R) impact the dynamics of the LIF model?
6. Determine how the firing rate of the LIF model varies with input I. Plot the firing rate versus I (i.e., plot the “f-I curve”).
8. **ADVANCED**. Simulate the [Izhikevich neuron](/Readings/Izhikevich_2003.pdf). Provide examples of two different types of spiking activity.